In [1]:
import coral as cr

# Overview

At some point, every synthetic biologist has to think about cloning - what method to use for stitching together the DNA that we want. In this example, we want to stitch together parts to create a plasmid that produces [GFP](https://en.wikipedia.org/wiki/Green_fluorescent_protein). The plasmid will be made up of these parts:
* A plasmid backbone so our DNA can be copied as our host, E. coli, grows
* A GFP gene sequence (DNA) so our bacteria can produce a fluorescent green protein
* A promoter sequence to turn the production of GFP
* A terminator sequence to define the end of [transcription](https://en.wikipedia.org/wiki/Transcription_(genetics))

If you don't know what these parts do, don't worry about it. The point is that we need to stitch them together, end-to-end, in this order:
* promoter - GFP - terminator - backbone

(The backbone must also be stitched to the promoter to create a circular DNA sequence.)

First, let's read in our DNA sequences.

In [11]:
# For simplicity's sake, we will download these parts from the iGEM parts registry
import requests
import StringIO

# This function makes it easy to grab DNA from the parts registry
# The StringIO section is just a hack to let us use Coral's parsers
def get_part(name):
    baseurl = 'http://parts.igem.org/fasta/parts/'
    part_fasta = requests.get(baseurl + name).text
    buff = StringIO.StringIO()
    buff.write(part_fasta)
    buff.seek(0)
    return cr.io.parsers.parse_fasta(buff)

# The plasmid backbone - pSB1A3
backbone = get_part('pSB1A3')
backbone.features.append(backbone.to_feature('pSB1A3'))
# The Tet promoter
promoter = get_part('BBa_R0040')
promoter.features.append(promoter.to_feature('Tet promoter'))
# The GFP gene
gfp = get_part('BBa_E0040')
gfp.features.append(gfp.to_feature('GFP', 'CDS'))
# A common terminator
terminator = get_part('BBa_B0015')
terminator.features.append(terminator.to_feature('Terminator'))

# Finally, we'll visualize our parts all together
(promoter + gfp + terminator + backbone).circularize().display()

## Gibson cloning
[Gibson cloning](https://en.wikipedia.org/wiki/Gibson_assembly) is 'scarless' method, which means that unlike many other cloning methods, we create DNA that looks exactly what we want, with nothing extra left over from the process.

In order to stitch our pieces together with Gibson cloning, we need to add homologies to them - matching sequences. The following images describes the process pretty well: ![Gibson cloning homology](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/Gibson_assembly_overview.png/330px-Gibson_assembly_overview.png)

Imagine that the green piece is our promoter and the black piece is the GFP gene. In order to stitch them together with Gibson cloning, we need to add a little bit of the GFP gene to the promoter piece, and possibly vice versa. All that really matters is that they share some overlapping DNA at their ends. In my lab, we choose these overlaps based on the binding strength of the overlap.

## Gibson cloning in Coral

Coral has Gibson cloning tools in the `cloning` module to make your life easier. But to understand what those tools do, let's try writing our own Gibson tool first.

The primary question is: how will we design our overlaps? By something called 'melting temperature', a quantifiable metric of the DNA.